In [2]:
import os
os.environ['DASHSCOPE_API_KEY'] = 'sk-a31a563bce7146bf8c8ab93ad5fea537'

from langchain_community.chat_models.tongyi import ChatTongyi
llm = ChatTongyi(name='qwen-plus')

In [16]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate

embedding = HuggingFaceBgeEmbeddings(
    model_name='./embeddings/bge-base-zh-v1.5',
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)

with open('斗破苍穹.txt', encoding='utf-8') as file:
    contents = file.read()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_text(contents)
vectorstroe = InMemoryVectorStore.from_texts(
    texts=splits, embedding=embedding
)
retriever = vectorstroe.as_retriever()

system_prompt = (
    "你是一个问答助手"
    "请使用以下上下文的检索片段回答用户的问题 "
    "如果你无法通过上下文获取到用户所需要的答案，则告诉用户你不知道。"
    "请将你的回答精简到三句以内，使你的回答尽可能简洁。"
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


No sentence-transformers model found with name ./embeddings/bge-base-zh-v1.5. Creating a new one with mean pooling.


In [17]:
rag_chain.invoke({'input': '萧炎和魂天帝谁厉害？'})

{'input': '萧炎和魂天帝谁厉害？',
 'context': [Document(id='7bc735c1-4d7b-4428-8e6b-98e8ef32569a', metadata={}, page_content='“挺狠的手段啊,为了完成帝品雏丹的最后一步,竟然造了这般杀戮…”萧炎轻声道。\n\n“成王败寇,可不讲究过程。”魂天帝冷笑一声,盯着萧炎,道:“今日,便是我双方决战之时,你若失败,联军之中,必然鸡犬不留!”\n\n“你若败,魂族,我也会赶尽杀绝。”萧炎淡淡的道,对于魂族,他早便是恨之入骨,若是有机会的话,他决不会做那等伪善之人,当杀,则杀!\n\n“哈哈,好,那今日,便让我来试试,究竟是我魂族亡,还是你联军灭!”\n\n魂天帝仰天大笑,滔天的血气从其体内暴涌而出,瞬间便是化为无尽血海,血海之中所弥漫的力量,强得令人感到恐怖。\n\n“魂族之人,退后千里!”血海弥漫,魂天帝声音冰冷的喝道。\n\n听到喝声,那些魂族的强者,也是连忙后退,两名斗帝强者的交手,足以毁天灭地,若是被绞入其中,就算是斗圣巅峰的强者,必然也是当场陨落,而且谁都明白,这场战争的真正胜负关键,乃是萧炎以及魂天帝二人,两人的失败,决定着双方最后的结局!\n\n“萧炎,今日是本帝封帝之时,你可要让本帝战个痛快!”\n\n魂天帝血眸之中,血气滔天,他站在血海之中,此刻倒也是有着冲天豪气,此人虽然手段毒辣,但也算得上是枭雄般的人物,今日之战,两人之中,必然有其一陨落!\n\n此战,唯有尽全力相搏!\n\n“奉陪到底!”\n\n萧炎双眸中,同样是有着点点火热涌动,这一战,将会是斗气大陆万年之内,最为巅峰的一战!\n\n不论成败,此战,必将流传千古!\n\n望着几乎霎那间便是变得空空荡荡的天空,所有人都是紧闭呼吸,两人虽未动手,但那股气势,却是已如同山岳般的压在所有人的头上。\n\n“哗哗!”\n\n血海滔天,弥漫着天际,魂天帝眼中的猩红,仿佛也是在此刻变得浓郁了许多,这般对恃半晌,他脚步猛的一步踏出。\n\n“轰!”\n\n随着他这一步的踏出,整个天空,都是在此刻颤抖了起来,而那滔天血海也是疯狂翻涌,直接是化为数万丈庞大的血浪,狠狠的对着萧炎翻卷而去。\n\n在那高达数万丈的血浪之下,萧炎的身形,渺小得就如同蝼蚁一般。\n\n萧炎泛着绚丽光芒的双瞳,盯着那声势骇然的血浪,那之中所蕴

In [18]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "假设有一个聊天记录和最新的用户问题，其中可能涉及到聊天记录中的上下文。"
    "请根据这些信息，生成一个独立的问题，以便无需查看聊天记录即可理解。"
    "不要直接回答问题，如有必要，请对其进行重构，否则请原样返回。"
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)



In [19]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [20]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = '萧炎和魂天帝谁厉害'
answer = rag_chain.invoke({'input': question, 'chat_history': chat_history})
print(answer['answer'])

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=answer['answer'])
    ]
)

萧炎和魂天帝的实力相当，初期僵持不下。魂天帝后来通过吞噬血云增强了自身实力。具体情况不好说，但魂天帝占据了一定优势。


In [21]:
question = '萧炎不是有异火么，为啥魂天帝占有一定优势呢？'
answer = rag_chain.invoke({'input': question, 'chat_history': chat_history})
print(answer['answer'])

萧炎虽然有异火，但魂天帝通过吞噬血云增强了实力，整体实力上占据了优势。具体战斗细节未详细说明。


In [ ]:
from operator import itemgetter